In [110]:
import numpy as np
import torch 
from deepface import DeepFace
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
from skimage.transform import resize
import json
import os.path as osp
import os
from os import listdir
from os.path import isfile, join
import sys
sys.path.insert(0, '/home/kw2960/Common_sense/visualstorytelling')
sys.path.insert(0, '/home/kw2960')
from vist_api import vist
from datetime import datetime
print(torch.cuda.is_available())

True


In [111]:
vist_images_dir = '../data/vist/images'
vist_annotations_dir = '../data/vist/data/visualstorytelling'
sis_test = vist.Story_in_Sequence(vist_images_dir, vist_annotations_dir, ["test"])

sis's [test] loaded. It took 1.08 seconds.
Make mapping ...
Mapping for [Albums][Images][Stories][Sents] done.


In [112]:
len(sis_test.Albums)

1011

Use the Deepface API to get face & emotion recognition for each image and save as json file \
https://github.com/serengil/deepface

In [7]:
def emotion_recognition(images_dir, out_dir, split="train"):

    b = datetime.now()
    emotions = {"angry": 0, "disgust": 0, "fear": 0, "happy": 0, "sad": 0, "surprise": 0, "neutral": 0, "no_face": 0}
    files = [f for f in listdir(join(images_dir, split)) if isfile(join(images_dir, split, f))]
    for img_id in files:
        _id = img_id.split(".jpg")[0]
        

        img_file = join(images_dir, split, img_id)
        try:
            detection = DeepFace.analyze(
                    img_path=img_file, 
                    actions=['emotion'],
                    enforce_detection=True
            )
            emotions[detection["dominant_emotion"]] += 1
        except:
            detection = {"dominant_emotion": "no_face_detected"}
            emotions["no_face"] += 1

        json_object = json.dumps(detection)
        out_path = osp.join(out_dir, split, _id + '.json')
        with open(out_path, "w") as outfile:
            outfile.write(json_object)
    print('emotion [%s] loaded. It took %.2f seconds.' % (split, (datetime.now() - b).total_seconds()))
    print(emotions)
    return emotions


def emotion_statistics(out_dir, split="train"):
    b = datetime.now()
    emotions = {"angry": 0, "disgust": 0, "fear": 0, "happy": 0, "sad": 0, "surprise": 0, "neutral": 0, "no_face_detected": 0}
    files = [f for f in listdir(join(out_dir, split)) if isfile(join(out_dir, split, f))]
    for img_id in files:
        info = json.load(open(join(out_dir, split, img_id)))
        emotions[info["dominant_emotion"]] += 1
    print('emotion [%s] loaded. It took %.2f seconds.' % (split, (datetime.now() - b).total_seconds()))
    return emotions
        


    


In [5]:
vist_images_dir = '../data/vist/images'
out_dir = '../data/vist/emotion_annotations'
# emotion_recognition(vist_images_dir, out_dir, "test")
# emotions = emotion_recognition(vist_images_dir, out_dir, "train")

In [11]:
out_dir = '../data/vist/emotion_annotations'
emotions = emotion_statistics(out_dir, "test")
print(emotions)

emotion [test] loaded. It took 3.86 seconds.
{'angry': 972, 'disgust': 35, 'fear': 2008, 'happy': 3393, 'sad': 2274, 'surprise': 197, 'neutral': 2881, 'no_face_detected': 9315}


In [8]:
train_emotions = emotion_statistics(out_dir, "train")
print(train_emotions)

emotion [train] loaded. It took 81.58 seconds.
{'angry': 7053, 'disgust': 330, 'fear': 15465, 'happy': 23081, 'sad': 16190, 'surprise': 1424, 'neutral': 20224, 'no_face_detected': 83763}
